<a href="https://colab.research.google.com/github/Vite-Noiz/Machine-Learning-Course/blob/main/My%20HW/MiniP2/MiniP2_Q1_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import re
from collections import defaultdict
!gdown 1Qvc_qKO2G9-bOv9_GTg_J6ZhljTHRHHv
# Read Spam.csv file
data = []
with open('spam.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        if row:
            label = row[0]
            text = row[1]
            data.append((label, text))

# Text Cleaning
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

# make list
cleaned_data = [(label, clean_text(text)) for (label, text) in data]

Downloading...
From: https://drive.google.com/uc?id=1Qvc_qKO2G9-bOv9_GTg_J6ZhljTHRHHv
To: /content/spam.csv
100% 504k/504k [00:00<00:00, 134MB/s]


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 606-607: invalid continuation byte

In [ ]:
# Spliting Data
split_ratio = 0.8
split_idx = int(len(cleaned_data) * split_ratio)
train_data = cleaned_data[:split_idx]
test_data = cleaned_data[split_idx:]

In [ ]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_counts = defaultdict(int)  # num of class
        self.word_counts = defaultdict(lambda: defaultdict(int))  # num of word in each class
        self.class_probs = defaultdict(float)  # each class probility
        self.word_probs = defaultdict(lambda: defaultdict(float))  # probility of each word in class

    def train(self, train_data):
        # total class and word
        total_docs = len(train_data)
        vocabulary = set()

        for label, text in train_data:
            self.class_counts[label] += 1
            words = text.split()
            for word in words:
                self.word_counts[label][word] += 1
                vocabulary.add(word)

        # محاسبه احتمالات کلاس‌ها (با اسموثینگ لاپلاس)
        for label in self.class_counts:
            self.class_probs[label] = (self.class_counts[label] + 1) / (total_docs + len(self.class_counts))

        # محاسبه احتمالات کلمات در هر کلاس (با اسموثینگ لاپلاس)
        for label in self.class_counts:
            total_words_in_class = sum(self.word_counts[label].values())
            vocab_size = len(vocabulary)
            for word in vocabulary:
                count = self.word_counts[label][word] + 1  # اسموثینگ
                self.word_probs[label][word] = count / (total_words_in_class + vocab_size)

    def predict(self, text):
        words = text.split()
        max_prob = -float('inf')
        best_label = None
        # محاسبه احتمال لگاریتمی برای هر کلاس
        for label in self.class_probs:
            log_prob = np.log(self.class_probs[label])
            for word in words:
                if word in self.word_probs[label]:
                    log_prob += np.log(self.word_probs[label][word])
                else:
                    pass  # نادیده گرفتن کلمات ناشناخته
            if log_prob > max_prob:
                max_prob = log_prob
                best_label = label
        return best_label

# آموزش مدل
model = MultinomialNaiveBayes()
model.train(train_data)

In [ ]:
# افزودن بخش محاسبه ماتریس آشفتگی و معیارها
def evaluate_model(test_data, model):
    confusion_matrix = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}

    for true_label, text in test_data:
        pred_label = model.predict(text)

        # کلاس مثبت: spam، کلاس منفی: ham
        if true_label == 'spam':
            if pred_label == 'spam':
                confusion_matrix['TP'] += 1
            else:
                confusion_matrix['FN'] += 1
        else:  # true_label == 'ham'
            if pred_label == 'ham':
                confusion_matrix['TN'] += 1
            else:
                confusion_matrix['FP'] += 1

    # محاسبه معیارها
    accuracy = (confusion_matrix['TP'] + confusion_matrix['TN']) / len(test_data)

    # Precision (دقت مثبت)
    if confusion_matrix['TP'] + confusion_matrix['FP'] == 0:
        precision = 0
    else:
        precision = confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FP'])

    # Recall (احساس)
    if confusion_matrix['TP'] + confusion_matrix['FN'] == 0:
        recall = 0
    else:
        recall = confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FN'])

    # F1-Score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)

    return confusion_matrix, accuracy, precision, recall, f1

# فراخوانی تابع ارزیابی
conf_matrix, acc, prec, rec, f1 = evaluate_model(test_data, model)

In [ ]:
# Accuracy
import numpy as np
correct = 0
total = len(test_data)
for label, text in test_data:
    prediction = model.predict(text)
    if prediction == label:
        correct += 1

accuracy = (correct / total) * 100
print(f'Accuracy of Model is: {accuracy:.2f}%')

print("--- معیارهای ارزیابی ---")
print(f"- ماتریس آشفتگی:")
print(f"  TP: {conf_matrix['TP']} | FP: {conf_matrix['FP']}")
print(f"  FN: {conf_matrix['FN']} | TN: {conf_matrix['TN']}")
print(f"- دقت (Accuracy): {acc * 100:.2f}%")
print(f"- دقت مثبت (Precision): {prec * 100:.2f}%")
print(f"- احساس (Recall): {rec * 100:.2f}%")
print(f"- امتیاز F1: {f1 * 100:.2f}%")

Accuracy of Model is: 98.21%
--- معیارهای ارزیابی ---
- ماتریس آشفتگی:
  TP: 131 | FP: 6
  FN: 14 | TN: 964
- دقت (Accuracy): 98.21%
- دقت مثبت (Precision): 95.62%
- احساس (Recall): 90.34%
- امتیاز F1: 92.91%


In [ ]:
import csv
import re
import numpy as np
from collections import defaultdict

# خواندن داده‌ها با تشخیص خودکار کدک
def detect_encoding(file_path):
    import chardet
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# خواندن داده‌ها
data = []
file_path = 'spam.csv'
encoding = detect_encoding(file_path)
with open(file_path, 'r', encoding=encoding) as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        if row:
            label = row[0]
            text = row[1]
            data.append((label, text))

# تمیز کردن متن
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

cleaned_data = [(label, clean_text(text)) for (label, text) in data]

# تقسیم داده به آموزش و آزمون (تصادفی)
import random
random.seed(56)
random.shuffle(cleaned_data)
split_ratio = 0.8
split_idx = int(len(cleaned_data) * split_ratio)
train_data = cleaned_data[:split_idx]
test_data = cleaned_data[split_idx:]

# آموزش مدل (همان کد قبلی)
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_counts = defaultdict(int)
        self.word_counts = defaultdict(lambda: defaultdict(int))
        self.class_probs = defaultdict(float)
        self.word_probs = defaultdict(lambda: defaultdict(float))

    def train(self, train_data):
        total_docs = len(train_data)
        vocabulary = set()
        for label, text in train_data:
            self.class_counts[label] += 1
            words = text.split()
            for word in words:
                self.word_counts[label][word] += 1
                vocabulary.add(word)
        # محاسبه احتمالات با اسموثینگ
        for label in self.class_counts:
            self.class_probs[label] = (self.class_counts[label] + 1) / (total_docs + len(self.class_counts))
        for label in self.class_counts:
            total_words_in_class = sum(self.word_counts[label].values())
            vocab_size = len(vocabulary)
            for word in vocabulary:
                count = self.word_counts[label][word] + 1
                self.word_probs[label][word] = count / (total_words_in_class + vocab_size)

    def predict(self, text):
        words = text.split()
        max_prob = -float('inf')
        best_label = None
        for label in self.class_probs:
            log_prob = np.log(self.class_probs[label])
            for word in words:
                if word in self.word_probs[label]:
                    log_prob += np.log(self.word_probs[label][word])
            if log_prob > max_prob:
                max_prob = log_prob
                best_label = label
        return best_label

model = MultinomialNaiveBayes()
model.train(train_data)

# پیش‌بینی و محاسبه معیارها
def evaluate_model(test_data):
    y_true = []
    y_pred = []
    for label, text in test_data:
        prediction = model.predict(text)
        y_true.append(label)
        y_pred.append(prediction)

    # ماتریس آشفتگی
    classes = list(set(y_true))
    confusion_matrix = defaultdict(lambda: defaultdict(int))
    for true, pred in zip(y_true, y_pred):
        confusion_matrix[true][pred] += 1

    # محاسبه TP, TN, FP, FN (برای کلاس 'spam')
    TP = confusion_matrix['spam']['spam']
    FP = confusion_matrix['ham']['spam']
    FN = confusion_matrix['spam']['ham']
    TN = confusion_matrix['ham']['ham']

    # دقت (Accuracy)
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # دقت مثبت (Precision)
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0

    # بازیابی (Recall)
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    return confusion_matrix, accuracy, precision, recall

# اجرای تابع ارزیابی
confusion_matrix, accuracy, precision, recall = evaluate_model(test_data)


In [ ]:
# نمایش ماتریس آشفتگی
print("\nماتریس آشفتگی:")
print("----------------------------")
print(f"{'':<10} | {'پیش‌بینی Spam':<15} | {'پیش‌بینی Ham':<15}")
print("----------------------------")
for true_label in ['spam', 'ham']:
    row = f"{true_label:<10} | "
    for pred_label in ['spam', 'ham']:
        row += f"{confusion_matrix[true_label][pred_label]:<15} | "
    print(row)
print("----------------------------")

# نمایش معیارها
print(f"\nدقت (Accuracy): {accuracy * 100:.2f}%")
print(f"دقت مثبت (Precision): {precision * 100:.2f}%")
print(f"بازیابی (Recall): {recall * 100:.2f}%")


ماتریس آشفتگی:
----------------------------
           | پیش‌بینی Spam   | پیش‌بینی Ham   
----------------------------
spam       | 139             | 19              | 
ham        | 8               | 949             | 
----------------------------

دقت (Accuracy): 97.58%
دقت مثبت (Precision): 94.56%
بازیابی (Recall): 87.97%
